# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import warnings
warnings.filterwarnings("ignore")
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
with open('CityWethear.csv', encoding="utf-8-sig") as csvfile:
    csvresult = pd.read_csv(csvfile)
maxtemp =  csvresult[["maxtemp"]].apply(pd.to_numeric) 
maxtemp

,maxtemp
0,57.00
1,81.88
2,76.66
3,25.72
4,82.38
...,...
577,16.32
578,55.40
579,34.95
580,42.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
coordinates = csvresult[["lat", "lng"]]  
humidity = csvresult["humidity"].astype(float)

In [17]:
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(coordinates, weights = humidity, dissipating = False, max_intensity = 80, point_radius = 1))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
dreamspot = csvresult.loc[(csvresult['maxtemp'] > 70) & (csvresult['maxtemp'] < 80) & (csvresult['windspeed'] < 10) &(csvresult['cloud'] == 0)]
dreamcoordinates = dreamspot[["lat", "lng"]]
dreamspot['Hotel Name'] = ""
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(dreamcoordinates))
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
len(dreamspot)

9

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df = dreamspot


for index,row in hotel_df.iterrows():
    lat = row["lat"]
    lng = row["lng"]
    hotelcoordinate =  str(f"{lat}, {lng}")
    radius = 5000
    keyword = "Hotel"
    hoteltype = "Hotel"

    # set up a parameters dictionary
    params = {
        "location": hotelcoordinate,
        "keyword": keyword,
        "radius": radius,
        "type": hoteltype,
        "key": g_key
    }

    # base url
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params,verify=False)
    hotel_name = response.json()
    
    try:
        hotelname = hotel_name["results"][0]["name"]
    except:
        hotelname = "there was no hotel around"
    print(hotelname)


Amari Galle Sri Lanka
Hotel da Canoa
Gateway Lifestyle Casino
Keingtawan Resort
there was no hotel around
Hotel Kyriad Muraya
iGowati Lodge
Pedrim Hotel
Eden Hotel


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in dreamspot.iterrows()]
coordinates = dreamspot[["lat", "lng"]]

In [23]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(coordinates,
    info_box_content=[f"Hotel Name: {hotel}" for hotel in hotel_info])
fig.add_layer(markers)
fig.add_layer(gmaps.heatmap_layer(dreamcoordinates))


# Display Map
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))